# Tutorial on the SNPObj Functionalities

In [25]:
import logging
import os
import sys
import numpy as np

dir = os.path.abspath('../../../snputils/')
if not dir in sys.path: sys.path.append(dir)

from snputils.snp.io.read.vcf import VCFReader
from snputils.snp.io.write import VCFWriter

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

### 1. Read VCF file into a SNPObj

Following, we show how to read a VCF file into a SNPObject. The SNPObj encapsulates similar attributes found in a VCF file, including:

* **calldata_gt:** An array containing the genotypes.
* **samples:** An array containing the sample IDs, or None if no samples are available.
* **variants_ref:** An array containing the reference of each SNP.
* **variants_alt:** An array containing the alternate alleles of each SNP.
* **variants_chrom:** An array containing the chromosome of each SNP.
* **variants_filter_pass:** An array containing flag for passed/failed control checks for each SNP.
* **variants_id:** An array containing SNP IDs.
* **variants_pos:** An array containing the position of each SNP in the chromosome.
* **variants_qual:** An array containing the Phred-scaled quality score for each SNP.

In [2]:
# Define path to VCF
query_path = '../../data/subset.vcf'

# Read VCF into SNPObj
reader = VCFReader(query_path)
snpobj = reader.read()

print("Attributes of the SNPObject:", snpobj.keys())

INFO:snputils.snp.io.read.vcf:Reading ../../data/subset.vcf
INFO:snputils.snp.io.read.vcf:Finished reading ../../data/subset.vcf
Attributes of the SNPObject: ['calldata_gt', 'samples', 'variants_ref', 'variants_alt', 'variants_chrom', 'variants_filter_pass', 'variants_id', 'variants_pos', 'variants_qual']


### 2. Accessing and Modifying Attributes

You can access and modify attributes using attribute or dictionary access:

In [3]:
# Accessing the original sample IDs
samples = snpobj.samples
print("Old sample IDs:", samples)

# Updating sample IDs using attribute access
snpobj.samples = ['sample_1', 'sample_2', 'sample_3', 'sample_4']
print("New sample IDs (attribute access):", snpobj.samples)

# Updating sample IDs using dictionary-like access
snpobj['samples'] = ['sample_1', 'sample_2', 'sample_3', 'sample_4']
print("New sample IDs (dictionary access):", snpobj['samples'])

Old sample IDs: ['HG00096' 'HG00097' 'HG00099' 'HG00100']
New sample IDs (attribute access): ['sample_1', 'sample_2', 'sample_3', 'sample_4']
New sample IDs (dictionary access): ['sample_1', 'sample_2', 'sample_3', 'sample_4']


### 3. Retrieve Number of Samples and SNPs

In [4]:
# Retrieving the number of samples from the SNPObject
n_samples = snpobj.n_samples()
print("Number of samples:", n_samples)

# Retrieving the number of SNPs from the SNPObject
n_snps = snpobj.n_snps()
print("Number of SNPs:", n_snps)

Number of samples: 4
Number of SNPs: 976599


### 4. Obtain Unique Chromosome Names

The `unique_chrom()` method retrieves unique chromosome names from the `variants_chrom` attribute, preserving their order of appearance.

In [5]:
print("Unique chromosomes:", snpobj.unique_chrom())

Unique chromosomes: ['21']


### 5. Change Chromosome Nomenclature

The `rename_chrom` method offers flexibility in modifying chromosome notations within genomic datasets. It supports changing between several typical naming conventions as well as applying custom regex transformations to suit specific data standardization needs.

**Typical Chromosome Notations**

1. **'chr':** This is a commonly used format where chromosomes are prefixed with 'chr', e.g., 'chr1', 'chr2'.
2. **'chm':** Similar to 'chr' but with a different prefix, e.g., 'chm1', 'chm2'.
3. **'chrom':** A more verbose prefix, e.g., 'chrom1', 'chrom2'.
4. **Plain:** Uses just the chromosome number or letter without a prefix, e.g., '1', '2'.

Below are examples demonstrating how to switch between different chromosome naming conventions and apply a custom regex transformation.

In [6]:
# Example of renaming chromosomes using predefined patterns
# This converts chromosome notations from 'chr' format to 'chm' format.
print("Chromosomes before renaming:", snpobj.unique_chrom())  # Display first 5 chromosomes
snpobj = snpobj.rename_chrom(inplace=False)
print("Chromosomes after renaming:", snpobj.unique_chrom())  

Chromosomes before renaming: ['21']


Chromosomes after renaming: ['chr21']


In [7]:
snpobj = snpobj.convert_chromosome_format(from_format='chr', to_format='plain', inplace=False)
print("Chromosomes after renaming:", snpobj.unique_chrom()) 

Chromosomes after renaming: ['21']


In [8]:
print("Unique chromosomes before renaming:", snpobj.unique_chrom())
snpobj = snpobj.rename_chrom(to_replace=['21'], value=['chr21'], regex=False)
print("Unique chromosomes after renaming:", snpobj.unique_chrom())

Unique chromosomes before renaming: ['21']


Unique chromosomes after renaming: ['chr21']


In [9]:
print("Unique chromosomes before renaming:", snpobj.unique_chrom())
snpobj = snpobj.rename_chrom(to_replace={'chr21' : '21'}, regex=False)
print("Unique chromosomes after renaming:", snpobj.unique_chrom())

Unique chromosomes before renaming: ['chr21']
Unique chromosomes after renaming: ['21']


### 6. Rename missing values

The `rename_missings()` method can be used to Replace missing values in the `calldata_gt` field. By default, missing values codified as "-1" are replaced with ".". If inplace=False, the modified object is returned. Otherwise, the operation is done in-place and None is returned. By default, inplace=False.

In [10]:
print(np.unique(snpobj['calldata_gt']))
snpobj = snpobj.rename_missings(before=".", after=-1)
print(np.unique(snpobj['calldata_gt']))

[0 1]
[0 1]


### 7. Filter SNPObj by SNPs

The `filter_snps()` method can be used to filter SNPs according to `chromosome name` and/or `SNP position`. It can also filter based on specific `SNP indexes`. It can either include or exclude SNPs that match the specified criteria. If inplace=False, the modified object is returned. Otherwise, the operation is done in-place and None is returned. By default, inplace=False. 

* Include SNPs at positions in the range (500k, 600k) from chromosomes '21':

In [11]:
print("SNPs before filtering:", len(snpobj.variants_pos))
snpobj = snpobj.filter_snps(chrom=['21'], pos=list(range(5000000, 600000)), include=True)
print("SNPs after filtering:", len(snpobj.variants_pos))

SNPs before filtering: 976599
SNPs after filtering: 976599


* Exclude all SNPs from chromosome 1:

In [12]:
print("Unique chromosomes before filtering:", snpobj.unique_chrom())
snpobj2 = snpobj.filter_snps(chrom='21', include=False)
print("Unique chromosomes after filtering:", snpobj2.unique_chrom())

Unique chromosomes before filtering: ['21']
Unique chromosomes after filtering: []


* Include the SNPs at indexes [0, 2, 4]:

In [13]:
print("Unique variants before filtering:", len(snpobj.variants_pos))
snpobj2 = snpobj.filter_snps(indexes=[0, 2, 4], include=True)
print("Unique variants after filtering:", len(snpobj2.variants_pos))

Unique variants before filtering: 976599
Unique variants after filtering: 3


### 8.  Filter SNPObj by samples

The `filter_samples()` method can be used to filter samples according to `sample names`, or based on `sample indexes`. It can either include or exclude samples that match the specified criteria. 

* Exclude samples 'sample_3' and 'sample_4':

In [15]:
print("Samples before filtering:", snpobj.samples)
snpobj2 = snpobj.filter_samples(samples=['sample_3', 'sample_4'], include=False)
print("Samples after filtering:", snpobj2.samples)

Samples before filtering: ['sample_1', 'sample_2', 'sample_3', 'sample_4']
Samples after filtering: ['sample_1' 'sample_2']


* Exclude samples at indexes [0, 3]:

In [17]:
print("Samples before filtering:", snpobj.samples)
snpobj2 = snpobj.filter_samples(indexes=[0, 3], include=False)
print("Samples after filtering:", snpobj2.samples)

Samples before filtering: ['sample_1', 'sample_2', 'sample_3', 'sample_4']
Samples after filtering: ['sample_2' 'sample_3']


### 9. Subset to common SNPs with another SNPObj

The ``subset_to_common_snps()`` method can be used to subset a SNPObj to the common markers (i.e., SNPs at the 
        same CHROM, POS, REF and ALT) with another SNPObj.

In [18]:
snpobj1 = VCFReader(query_path).read()
snpobj2 = VCFReader(query_path).read()

n_snps = snpobj1.n_snps()
snpobj1 = snpobj1.subset_to_common_snps(snpobj2)

print(f'\n{snpobj1.n_snps()}/{n_snps} SNPs are common markers')

INFO:snputils.snp.io.read.vcf:Reading ../../data/subset.vcf
INFO:snputils.snp.io.read.vcf:Finished reading ../../data/subset.vcf
INFO:snputils.snp.io.read.vcf:Reading ../../data/subset.vcf
INFO:snputils.snp.io.read.vcf:Finished reading ../../data/subset.vcf

976599/976599 SNPs are common markers


### 10. Clean SNPObj

In [20]:
reader = VCFReader(query_path)
query = reader.read()

# Define path to reference VCF data
reference_path = query_path

reader = VCFReader(reference_path)
reference = reader.read()

INFO:snputils.snp.io.read.vcf:Reading ../../data/subset.vcf
INFO:snputils.snp.io.read.vcf:Finished reading ../../data/subset.vcf
INFO:snputils.snp.io.read.vcf:Reading ../../data/subset.vcf
INFO:snputils.snp.io.read.vcf:Finished reading ../../data/subset.vcf


#### 10.1 Remove strand-ambiguous SNPs

The `remove_strand_ambiguous_snps()` method searches and removes strand-ambiguous SNPs. A/T, T/A, C/G, and G/C pairs are strand-ambiguous because their components are complementary, making it difficult to determine the DNA strand. If inplace=False, the modified object is returned. Otherwise, the operation is done in-place and None is returned. By default, inplace=False. 

In [21]:
query2 = query.remove_strand_ambiguous_snps(inplace=False)
print(query.n_snps()-query2.n_snps(), "ambiguous SNPs removed.")

INFO:snputils.snp.genobj.snpobj:35183 ambiguities of A-T type.
INFO:snputils.snp.genobj.snpobj:35105 ambiguities of T-A type.
INFO:snputils.snp.genobj.snpobj:39334 ambiguities of C-G type.
INFO:snputils.snp.genobj.snpobj:38992 ambiguities of G-C type.
INFO:snputils.snp.genobj.snpobj:148614 strand-ambiguous SNPs in total.
148614 ambiguous SNPs removed.


#### 10.2 Correct SNP flips

The ``correct_snp_flips`` method searches and corrects SNP flips between the current snpobj (query) and another snpobj (reference). An SNP flip occurs when the 'variants_ref' and 'variants_alt' fields are swapped in the current object (query) and the provided snpobj (reference). SNP flips are corrected by swapping the 'variants_ref' and 'variants_alt'  fields in the query, as well as their respective 0s and 1s in the 'calldata_gt' field. If inplace=False, the modified object is returned. Otherwise, the operation is done in-place and None is returned. By default, inplace=False. 

In [22]:
query.correct_snp_flips(reference, inplace=True)

INFO:snputils.snp.genobj.snpobj:Matching ref and alts are: 976599, 976599
INFO:snputils.snp.genobj.snpobj:Number of ambiguous is 0
INFO:snputils.snp.genobj.snpobj:Correcting 148614 SNP flips.


#### 10.3 Remove Mismatching SNPs

In [23]:
query2 = query.remove_mismatching_snps(reference, inplace=False)
print(query.n_snps()-query2.n_snps(), "mismatching SNPs removed.")

INFO:snputils.snp.genobj.snpobj:976599 SNPs at the same position.
INFO:snputils.snp.genobj.snpobj:148614 mismatching SNPs in total.
148614 mismatching SNPs removed.


### 11. Write SNPObj into a VCF file

The `VCFWriter()` can be used to write a SNPObj into a VCF file. If `chrom_partition=True` when calling `write()` method, then individual VCF files are generated for each chromosome. If False, a single VCF file containing data for all chromosomes is created. By default, `chrom_partition=False`.

In [24]:
# Define path to output VCF
output_path = '../../data/trush.vcf'

reader = VCFReader(query_path)
snpobj = reader.read()
writer = VCFWriter(snpobj, output_path)
writer.write(chrom_partition=False)

INFO:snputils.snp.io.read.vcf:Reading ../../data/subset.vcf
INFO:snputils.snp.io.read.vcf:Finished reading ../../data/subset.vcf
